# 1 How many columns are there?

In [1]:
import pandas as pd

In [220]:
data = pd.read_parquet('c:/Users/venu_reddy/Downloads/yellow_tripdata_2023-01.parquet')

In [221]:
data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [222]:
data.shape

(3066766, 19)

####  jan we 19 columns 

# 2 Computing duration

In [223]:
data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [224]:
data["time"] = data["tpep_dropoff_datetime"] - data["tpep_pickup_datetime"]

In [225]:
data["time"] = data["time"].apply(lambda x : x.seconds/60 )

In [226]:
data["time"].mean()

np.float64(15.653030467056613)

In [227]:
data["time"].std()

np.float64(41.62919110966266)

In [228]:
# i seen some thing difference we are getting 

In [229]:
data['duration'] = data.tpep_dropoff_datetime - data.tpep_pickup_datetime
data['duration'] = data.duration.dt.total_seconds() / 60

In [230]:
data["duration"].mean()

np.float64(15.668995167330452)

In [231]:
data["duration"].std()

np.float64(42.59435124195458)

# 3 .  Dropping outliers

In [232]:
len(data[(data["duration"] >= 1) & (data["duration"] <= 60)]) / len(data) *100

98.1220282212598

In [233]:
data = data[(data["duration"] >= 1) & (data["duration"] <= 60)]

# 4 . One-hot encoding

In [234]:
from sklearn.feature_extraction import DictVectorizer

In [183]:
categorical = ['PULocationID', 'DOLocationID']
data[categorical] = data[categorical].astype(str)
train_dicts = data[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(f'The shape of the  matrix {X_train.shape}')



The shape of the  matrix (3009173, 515)


# 5 . Training a Model

In [184]:
from sklearn.linear_model import LinearRegression

y_train = data["duration"].values


In [185]:
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [186]:
model = LinearRegression()
model.fit(X_train,y_train)


LinearRegression()

In [187]:
y_prd = model.predict(X_train)

In [193]:

from sklearn.metrics  import mean_squared_error
mean_squared_error(y_train, y_prd, squared=False)

np.float64(7.649261934246708)

In [241]:
#  using pipe 

In [243]:
from sklearn.pipeline  import Pipeline

pipe = Pipeline([('vector',DictVectorizer()),('model',LinearRegression())])

In [244]:
pipe.fit(train_dicts,y_train)

Pipeline(steps=[('vector', DictVectorizer()), ('model', LinearRegression())])

In [245]:
y_prd = pipe.predict(train_dicts)

In [246]:
from sklearn.metrics  import mean_squared_error
mean_squared_error(y_train, y_prd, squared=False)

np.float64(7.649261934246708)

# Evaluating the model

In [194]:
data_feb = pd.read_parquet('c:/Users/venu_reddy/Downloads/yellow_tripdata_2023-02.parquet')

In [211]:


data_feb["duration"] = data_feb.tpep_dropoff_datetime - data_feb.tpep_pickup_datetime
data_feb["duration"] = data_feb.duration.dt.total_seconds() / 60


data_feb = data_feb[(data_feb["duration"] >= 1) & (data_feb["duration"] <= 60)].copy()
data_feb[categorical] = data_feb[categorical].astype('str')
val_dicts = data_feb[categorical].to_dict(orient='records')
X_feb = dv.transform(val_dicts)
y_feb = data_feb["duration"]
y_pred_feb  =model.predict(X_feb)
mean_squared_error(y_feb, y_pred_feb,squared=False)

np.float64(7.811818474238467)

In [207]:
mean_squared_error(y_feb, y_pred_feb,squared=False)

np.float64(7.811818474238467)

In [242]:
y_pred_feb = pipe.predict(val_dicts)
mean_squared_error(y_feb, y_pred_feb,squared=False)

np.float64(7.811818474238467)